<a href="https://colab.research.google.com/github/fatimasadiq/NLP_Module_Project_hatespeechdetection/blob/main/Spring_NLP_Bert_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip gdrive/My\ Drive/askfmbert.zip

Archive:  gdrive/My Drive/askfmbert.zip
  inflating: berttrainsenti.csv      
  inflating: __MACOSX/._berttrainsenti.csv  
  inflating: berttestsenti.csv       
  inflating: __MACOSX/._berttestsenti.csv  


In [ ]:
!git clone https://github.com/aub-mind/arabert

Cloning into 'arabert'...
remote: Enumerating objects: 556, done.
remote: Counting objects: 100% (342/342), done.
remote: Compressing objects: 100% (245/245), done.
remote: Total 556 (delta 182), reused 244 (delta 91), pack-reused 214
Receiving objects: 100% (556/556), 9.16 MiB | 23.32 MiB/s, done.
Resolving deltas: 100% (305/305), done.


In [ ]:
!pip install transformers==4.1.1

     |████████████████████████████████| 1.5 MB 5.4 MB/s 
     |████████████████████████████████| 2.9 MB 32.2 MB/s 
     |████████████████████████████████| 895 kB 31.4 MB/s 


In [ ]:
from transformers import AutoTokenizer, AutoModel

In [ ]:
!pip install torch==1.9.0

In [ ]:
!pip install torchtext==0.4.0

     |████████████████████████████████| 53 kB 1.2 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0


In [ ]:
!pip install sentencepiece==0.1.94

     |████████████████████████████████| 1.1 MB 5.4 MB/s 


In [ ]:
!pip install torchtext==0.4.0

In [ ]:
# Libraries
import matplotlib.pyplot as plt
import pandas as pd
import torch
# Preliminaries
from torchtext.data import Field, TabularDataset, BucketIterator, Iterator
# Models
import torch.nn as nn
from csv import reader
import numpy as np

In [ ]:
import transformers
# model = transformers.BertForMaskedLM.from_pretrained("TurkuNLP/bert-base-finnish-uncased-v1")
from transformers import AlbertTokenizer, TFAlbertModel

# tokenizer = AlbertTokenizer.from_pretrained('albert-base-v1')
# model = TFAlbertModel.from_pretrained("albert-base-v1")

#tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
#model = transformers.BertModel.from_pretrained("bert-base-multilingual-uncased")

#tokenizer = transformers.BertTokenizer.from_pretrained('Hate-speech-CNERG/dehatebert-mono-english')
#model = transformers.BertModel.from_pretrained("Hate-speech-CNERG/dehatebert-mono-english")

tokenizer = transformers.BertTokenizer.from_pretrained('GroNLP/hateBERT')
model = transformers.BertModel.from_pretrained("GroNLP/hateBERT")
# from transformers import RobertaTokenizer, RobertaModel
# tokenizer = RobertaTokenizer.from_pretrained('Hate-speech-CNERG/dehatebert-mono-english')
# model = RobertaModel.from_pretrained('Hate-speech-CNERG/dehatebert-mono-english')

model.eval()
if torch.cuda.is_available():
    model = model.cuda()
# tokenizer = transformers.BertTokenizer.from_pretrained("TurkuNLP/bert-base-finnish-uncased-v1")

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/151 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
bert=model

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [ ]:
train = pd.read_csv("/content/sprintberttrain.csv", sep=',')
valid = pd.read_csv("/content/springberttest.csv", sep=',')

In [ ]:
#Data Loading 
train=pd.read_csv('/content/sprintberttrain.csv') #, error_bad_lines=False, delimiter=";", usecols=["label","q_a"]) #error_bad_lines=False 
train.head(5)

,label,posts
0,0,lol
1,0,idnt even kno iz
2,0,hell yea ikan wham ta da left nd ta da rite bi...
3,0,cooling
4,0,vanilla


In [ ]:
valid=pd.read_csv('/content/springberttest.csv') #, error_bad_lines=False, delimiter=";", usecols=["label","q_a"]) #error_bad_lines=False 
valid.head(5)

,label,posts
0,0,haha jk
1,0,really thanks haha
2,0,NaN
3,0,rawr
4,0,sure like tell wht u wnna know like wht use


In [ ]:
len(train)

7633

In [ ]:
# Model parameter
MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

# Fields

label_field = Field(sequential=False, use_vocab=False, batch_first=True, dtype=torch.float)
text_field = Field(use_vocab=False, tokenize=tokenizer.encode, lower=False, include_lengths=False, batch_first=True,
                   fix_length=MAX_SEQ_LEN, pad_token=PAD_INDEX, unk_token=UNK_INDEX)
fields = [('label', label_field),('posts', text_field)]

# TabularDataset

train, valid= TabularDataset.splits( path="", train='/content/sprintberttrain.csv', validation='/content/springberttest.csv',
                                            format='CSV', fields=fields, skip_header=True)


# Iterators

train_iter = BucketIterator(train, batch_size=4, sort_key=lambda x: len(x.posts),
                            device=device, train=True, sort=True, sort_within_batch=True)
valid_iter = BucketIterator(valid, batch_size=4, sort_key=lambda x: len(x.posts),
                            device=device, train=True, sort=True, sort_within_batch=True)
test_iter = Iterator(valid, batch_size=4, device=device, train=False, shuffle=False, sort=False)

In [ ]:
train

In [ ]:
train_iter

In [ ]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(768,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output


In [ ]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 4
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 114,606,849 trainable parameters


In [ ]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 5,124,609 trainable parameters


In [ ]:
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
rnn.weight_ih_l2
rnn.weight_hh_l2
rnn.bias_ih_l2
rnn.bias_hh_l2
rnn.weight_ih_l2_reverse
rnn.weight_hh_l2_reverse
rnn.bias_ih_l2_reverse
rnn.bias_hh_l2_reverse
rnn.weight_ih_l3
rnn.weight_hh_l3
rnn.bias_ih_l3
rnn.bias_hh_l3
rnn.weight_ih_l3_reverse
rnn.weight_hh_l3_reverse
rnn.bias_ih_l3_reverse
rnn.bias_hh_l3_reverse
out.weight
out.bias


In [ ]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.BCEWithLogitsLoss()

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def binary_f1score(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
def get_classification_metric(testy, probs):
    from sklearn.metrics import precision_recall_curve
    precision, recall, thresholds = precision_recall_curve(testy, probs)
    # convert to f score
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    
    return fscore

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    #epoch_f1=0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.posts).squeeze(1)
        # print(predictions)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        #f1score=get_classification_metric(predictions,batch.label)#.cpu().detach().numpy())
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
       # epoch_f1+=f1score.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)#,epoch_f1 / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    #epoch_f1=0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.posts).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)
            #f1score=get_classification_metric(predictions,batch.label)


            epoch_loss += loss.item()
            epoch_acc += acc.item()
            #epoch_f1+=f1score.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)#,epoch_f1 / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 2

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iter, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '/content/gdrive/MyDrive/Colab Notebooks/bert/hateBERT.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 9m 29s
	Train Loss: 0.012 | Train Acc: 99.88%
	 Val. Loss: 0.000 |  Val. Acc: 100.00%
Epoch: 02 | Epoch Time: 9m 28s
	Train Loss: 0.011 | Train Acc: 99.88%
	 Val. Loss: 0.000 |  Val. Acc: 100.00%


In [ ]:
model.load_state_dict(torch.load('/content/gdrive/MyDrive/Colab Notebooks/bert/hateBERT.pt'))

test_loss, test_acc = evaluate(model, test_iter, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.000 | Test Acc: 100.00%


In [ ]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

In [ ]:
def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:512-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    sentiment=np.round(prediction.item())
    return sentiment

In [ ]:
predict_sentiment(model, tokenizer, "you are fuck")

0.0